In [54]:
!pip install -U sentence-transformers

In [55]:
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer

from sklearn.decomposition import PCA
from sklearn.metrics import DistanceMetric

import matplotlib.pyplot as plt
import matplotlib as mpl

In [56]:
model_name="Sakil/sentence_similarity_semantic_search"
model = SentenceTransformer(model_name)

In [ ]:
class Article:
    def __init__(self, id, title, author, text, text_emb, list_prop, list_prop_emb):
        self.id = id
        self.title = title
        self.author = author
        self.text = text
        self.text_emb = text_emb
        self.list_prop_emb = list_prop_emb
        self.list_prop = list_prop

    def __repr__(self):
        return f"Article(id={self.id}, title={self.title}, author={self.author})"

    def get_summary(self, num_sentences=3):
        """Return a short summary of the text (first `num_sentences` sentences)."""
        sentences = self.text.split(".")
        return ". ".join(sentences[:num_sentences]) + "."

In [119]:
articole_list = []
data = pd.read_csv('test.csv')

for i, articol in data.iterrows():
    print("ARTICOL", i)
    # Verifică dacă textul articolului nu este gol sau NaN
    if pd.isna(articol['text']):
        continue

    # Extragem propozițiile din text
    sentence_list = [sentence.strip() for sentence in articol['text'].split(".") if sentence]

    # Calculăm embedding-ul pentru întregul text al articolului
    text_emb = model.encode(articol['text'])

    # Calculăm embedding-urile pentru fiecare propoziție
    emb_list = model.encode(sentence_list)

    # Creăm un obiect Article și îl adăugăm în lista articolelor
    articol_final = Article(
        id=articol['id'],
        title=articol['title'],
        author=articol['author'],
        text=articol['text'],
        text_emb=text_emb,
        list_prop=sentence_list,
        list_prop_emb=emb_list
    )
    articole_list.append(articol_final)


ARTICOL 0
ARTICOL 1
ARTICOL 2
ARTICOL 3
ARTICOL 4
ARTICOL 5
ARTICOL 6
ARTICOL 7
ARTICOL 8
ARTICOL 9
ARTICOL 10
ARTICOL 11
ARTICOL 12
ARTICOL 13
ARTICOL 14
ARTICOL 15
ARTICOL 16
ARTICOL 17
ARTICOL 18
ARTICOL 19
ARTICOL 20
ARTICOL 21
ARTICOL 22
ARTICOL 23
ARTICOL 24
ARTICOL 25
ARTICOL 26
ARTICOL 27
ARTICOL 28
ARTICOL 29
ARTICOL 30
ARTICOL 31
ARTICOL 32
ARTICOL 33
ARTICOL 34
ARTICOL 35
ARTICOL 36
ARTICOL 37
ARTICOL 38
ARTICOL 39
ARTICOL 40
ARTICOL 41
ARTICOL 42
ARTICOL 43
ARTICOL 44
ARTICOL 45
ARTICOL 46
ARTICOL 47
ARTICOL 48
ARTICOL 49
ARTICOL 50
ARTICOL 51
ARTICOL 52
ARTICOL 53
ARTICOL 54
ARTICOL 55
ARTICOL 56
ARTICOL 57
ARTICOL 58
ARTICOL 59
ARTICOL 60
ARTICOL 61
ARTICOL 62
ARTICOL 63
ARTICOL 64
ARTICOL 65
ARTICOL 66
ARTICOL 67
ARTICOL 68
ARTICOL 69
ARTICOL 70
ARTICOL 71
ARTICOL 72
ARTICOL 73
ARTICOL 74
ARTICOL 75
ARTICOL 76
ARTICOL 77
ARTICOL 78
ARTICOL 79
ARTICOL 80
ARTICOL 81
ARTICOL 82
ARTICOL 83
ARTICOL 84
ARTICOL 85
ARTICOL 86
ARTICOL 87
ARTICOL 88
ARTICOL 89
ARTICOL 90
ARTICOL 9

In [117]:
from sentence_transformers import util

# Exemplu de query
query = "a person is a very very old hero"
query_embedding = model.encode([query])

# Dicționar pentru a stoca cele mai similare propoziții pentru fiecare articol
bestSentancePerArticle = {}

for articol in articole_list:
    # Calculăm similaritatea între fiecare propoziție și query
    cos_simProp = util.cos_sim(articol.list_prop_emb, query_embedding)

    # Obținem scorurile de similaritate și le sortăm în ordine descrescătoare
    sorted_similarities = sorted(enumerate(cos_simProp[0]), key=lambda x: x[1], reverse=True)

    # Salvăm cea mai similară propoziție pentru fiecare articol (index + scor)
    bestSentancePerArticle[articol.id] = sorted_similarities[0]

# Sortăm dicționarul `bestSentancePerArticle` în funcție de scorurile de similaritate, descrescător
sorted_best_sentences = sorted(bestSentancePerArticle.items(), key=lambda x: x[1][1].item(), reverse=True)

# Afișăm primele 5 cele mai similare propoziții pentru query
print("Top-5 most similar sentences to the query:")
for i, (article_id, (index, score)) in enumerate(sorted_best_sentences[:5]):
    articol = next(art for art in articole_list if art.id == article_id)
    print(f"Similarity score: {score.item():.4f}")
    print(f"Article title: {articol.title}")
    print(f"Most similar sentence: {articol.list_prop[index]}")
    print("-" * 40)



Top-5 most similar sentences to the query:
Similarity score: 0.4493
Article title: A Jewish Player’s 1914 Baseball Card Triggers a $125,000 Dispute - The New York Times
Most similar sentence: More than a hundred years after Guy Zinn last appeared in a   game, his baseball card is causing a commotion
----------------------------------------
Similarity score: 0.4119
Article title: Trump is USA's antique hero. Clinton will be next president
Most similar sentence: Trump is USA's antique hero
----------------------------------------
Similarity score: 0.3767
Article title: Charles Koch Makes Massive Donation to Historically Black Colleges - Breitbart
Most similar sentence: Billionaire Charles Koch, who is a frequent conservative political donor, has made a massive contribution to historically black colleges across the United States
----------------------------------------
Similarity score: 0.3666
Article title: nan
Most similar sentence: This is the biggest story not being reported
---------